# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
# import required libraries
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import pymysql

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
driver  = 'mysql+pymysql:'
user    = 'root'
password= 'Tarradellas90'
ip      = ':3306'
database= 'orders'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
conn = create_engine(connection_string)

In [4]:
df_order = pd.read_sql('SELECT * FROM orders', conn)
original = df_order
df_order.head(5)

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: YES)")
(Background on this error at: http://sqlalche.me/e/e3q8)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# your code here
for col in df_order.columns:
    print(col)
    uni = df_order[col].unique()
    if len(uni)<50:
        print(col)
        print('\t',uni)

In [ ]:
#Sub Problem 1: How to aggregate the amount_spent for unique customers?
idx = df_order[['CustomerID','amount_spent']]
df_order_amount = idx.groupby('CustomerID').sum().sort_values('amount_spent', ascending=False)
df_order_amount.head(10)

In [ ]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
customer_type = ['Casual', 'Usual', 'Preferred', 'VIP']
customers_type= pd.cut(df_order_amount['amount_spent'], len(customer_type), labels=customer_type, retbins= True)
customers_type

In [ ]:
customer_type = ['Casual', 'Usual', 'Preferred', 'VIP']
df_order_amount['Customer Cat']= pd.qcut(df_order_amount['amount_spent'], len(customer_type), labels = customer_type)
df_order_amount

In [ ]:

cat_column = df_order_amount.drop(columns = ['amount_spent'])
cat_column

In [ ]:
df_order_label = pd.merge(df_order, cat_column, how='left', on = 'CustomerID')
df_order_label

In [ ]:
#Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?
df_customer_vip_preferred = df_order_label.loc[(df_order_label['Customer Cat'] == 'VIP') | (df_order_label['Customer Cat'] == 'Preferred')]
df_customer_vip_preferred.head(5)

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# your code here
df_customer_vip = df_order_label.loc[(df_order_label['Customer Cat'] == 'VIP')]
df_vips_by_country = df_customer_vip.groupby(['Country']).count()[['Customer Cat']]
df_vips_by_country

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here
df_vips_preferred_by_country =  df_order_label.groupby(['Country']).count()[['Customer Cat']]
df_vips_preferred_by_country